In [8]:
import json
import os
from azureml.core import Workspace, Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Comprobar las versiones de los paquetes
import pkg_resources
print("Versiones de paquetes:")
print(f"azureml-core: {pkg_resources.get_distribution('azureml-core').version}")
print(f"Python: {os.sys.version}")

Versiones de paquetes:
azureml-core: 1.48.0
Python: 3.10.16 (main, Mar 17 2025, 20:54:03) [MSC v.1943 64 bit (AMD64)]


In [ ]:
print("Cargando ID de suscripción de Azure...")
try:
    with open('id_CONFIDENTIAL.json', 'r') as id_file:
        mi = json.load(id_file)
        my_id = mi["my_id"]
    print("ID de suscripción cargado correctamente.")
except Exception as e:
    print(f"Error al cargar el ID de suscripción: {e}")
    print("Asegúrate de crear un archivo 'id_CONFIDENTIAL.json' con tu ID de suscripción de Azure.")
    print('Formato: {"my_id": "tu-id-de-suscripción-aquí"}')
    exit(1)

Cargando ID de suscripción de Azure...
ID de suscripción cargado correctamente.


In [11]:
# Nombre del grupo de recursos y ubicación
resource_group = "bankruptcy_predictor_rg_diego"  # Nombre actualizado
location = "centralindia"  # Puedes cambiar esto según tu preferencia

print(f"Creando workspace en Azure en {location}...")
# Crear workspace de Azure ML
try:
    ws = Workspace.create(
        name="bankruptcy_workspace_2nd",  # Nombre actualizado
        subscription_id=my_id,
        resource_group=resource_group,
        location=location,
        exist_ok=True  # No falla si ya existe
    )
    print("Workspace creado/cargado exitosamente.")
    
    # Verificar los detalles del workspace
    print("Detalles del workspace:")
    print(f"Nombre: {ws.name}")
    print(f"Región: {ws.location}")
    print(f"ID de recurso: {ws.id}")
except Exception as e:
    print(f"Error al crear el workspace: {e}")
    exit(1)

Creando workspace en Azure en centralindia...
Deploying KeyVault with name bankruptkeyvault1ecabe6e.
Deploying StorageAccount with name bankruptstorageeeaae05b6.
Deploying AppInsights with name bankruptinsightscaefcc4e.
Deployed AppInsights with name bankruptinsightscaefcc4e. Took 23.39 seconds.
Deployed KeyVault with name bankruptkeyvault1ecabe6e. Took 26.52 seconds.
Deploying Workspace with name bankruptcy_workspace_2nd.
Deployed StorageAccount with name bankruptstorageeeaae05b6. Took 31.82 seconds.
Deployed Workspace with name bankruptcy_workspace_2nd. Took 42.2 seconds.
Workspace creado/cargado exitosamente.
Detalles del workspace:
Nombre: bankruptcy_workspace_2nd
Región: centralindia
Error al crear el workspace: 'Workspace' object has no attribute 'id'


In [ ]:
# Verificar que el modelo exista
if not os.path.exists("model.pkl"):
    print("Error: No se encontró el archivo 'model.pkl'.")
    print("Ejecuta primero el script 'modelo.py' para entrenar y guardar el modelo.")
    exit(1)

# Registrar el modelo en Azure ML
print("Registrando modelo en Azure ML...")
model_name = "bankruptcy_model_2nd"
try:
    registered_model = Model.register(
        model_path="model.pkl",
        model_name=model_name,
        workspace=ws
    )
    print(f"Modelo registrado con ID: {registered_model.id}")
    print(f"Nombre del modelo: {registered_model.name}")
    print(f"Versión del modelo: {registered_model.version}")
except Exception as e:
    print(f"Error al registrar el modelo: {e}")
    exit(1)

Registrando modelo en Azure ML...
Registering model bankruptcy_model_2nd
Modelo registrado con ID: bankruptcy_model_2nd:1
Nombre del modelo: bankruptcy_model_2nd
Versión del modelo: 1


In [13]:
import json
import os
from azureml.core import Workspace, Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Comprobar las versiones de los paquetes
import pkg_resources
print("Versiones de paquetes:")
print(f"azureml-core: {pkg_resources.get_distribution('azureml-core').version}")
print(f"Python: {os.sys.version}")

Versiones de paquetes:
azureml-core: 1.48.0
Python: 3.10.16 (main, Mar 17 2025, 20:54:03) [MSC v.1943 64 bit (AMD64)]


In [14]:
# Cargar el umbral calculado
if not os.path.exists("umbral.json"):
    print("Error: No se encontró el archivo 'umbral.json'.")
    print("Ejecuta primero el script 'modelo.py' para calcular y guardar el umbral.")
    exit(1)

print("Cargando umbral de clasificación...")
with open("umbral.json", "r") as umb_file:
    umb = json.load(umb_file)
    umbral = umb["umbral"][0]
    print(f"Umbral cargado: {umbral}")

# Crear el script score.py para inferencia
print("Creando script de inferencia (score.py)...")

# Contenido del script score.py - VERSIÓN CORREGIDA CON MEJOR MANEJO DE ERRORES
scorepy = f"""
import json
import joblib
import numpy as np
import pandas as pd
import os
import traceback
from azureml.core.model import Model

def init():
    global model
    try:
        # Imprimimos información para depuración
        print("Python version:", os.sys.version)
        print("Current working directory:", os.getcwd())
        
        # Obtener la ruta del modelo
        model_path = Model.get_model_path('{model_name}')
        print(f"Ruta del modelo: {{model_path}}")
        
        # Verificar si el archivo existe
        if not os.path.exists(model_path):
            print(f"ERROR: El archivo del modelo no existe en {{model_path}}")
            raise FileNotFoundError(f"No se encuentra el modelo en {{model_path}}")
            
        # Cargar el modelo
        print("Cargando el modelo...")
        model = joblib.load(model_path)
        print(f"Modelo cargado exitosamente. Tipo: {{type(model)}}")
        
    except Exception as e:
        print(f"ERROR en init(): {{str(e)}}")
        print("Traceback completo:")
        traceback.print_exc()
        # Re-lanzar la excepción para que Azure ML pueda registrarla
        raise

def run(raw_data):
    try:
        # Parsear los datos JSON recibidos
        print("Recibiendo datos...")
        data = json.loads(raw_data)['data'][0]
        
        # Convertir a DataFrame
        print("Convirtiendo a DataFrame...")
        data = pd.DataFrame(data)
        print(f"Datos recibidos con forma: {{data.shape}}")
        
        # Realizar la predicción
        print("Realizando predicción...")
        result_proba = model.predict_proba(data)[:, 1].tolist()  # Probabilidad de clase positiva
        umbral = {umbral}
        result_finals = [1 if x > umbral else 0 for x in result_proba]
        
        print(f"Predicción completada. Resultados: {{result_finals}}")
        return json.dumps(result_finals)
    except Exception as e:
        error_message = str(e)
        print(f"ERROR en run(): {{error_message}}")
        print("Traceback completo:")
        traceback.print_exc()
        return json.dumps({{"error": error_message, "traceback": traceback.format_exc()}})
"""

# Escribir el archivo score.py
with open("score.py", "w") as file_score:
    file_score.write(scorepy)
    print("Script de inferencia creado: score.py")

Cargando umbral de clasificación...
Umbral cargado: 0.5500000000000002
Creando script de inferencia (score.py)...
Script de inferencia creado: score.py


In [ ]:
print("Configurando entorno de inferencia...")

virtual_env = Environment("bankruptcy_env_diego") 
conda_deps = CondaDependencies.create(
    python_version="3.10",  
    conda_packages=[
        'numpy=1.19.5',
        'pandas=1.2.4', 
        'scikit-learn=0.24.2' 
    ],
    pip_packages=[
        'azureml-defaults==1.41.0',
        'joblib==1.0.1'
    ]
)

virtual_env.python.conda_dependencies = conda_deps

# Configuración de inferencia
inference_config = InferenceConfig(
    environment=virtual_env,
    entry_script="score.py",
    source_directory=None
)

# Mostrar la configuración del entorno para verificar
print("Configuración del entorno:")
print(virtual_env.python.conda_dependencies.serialize_to_string())

Configurando entorno de inferencia...
Configuración del entorno:
# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.8 and later.
- python=3.8

- pip:
  - azureml-defaults~=1.48.0
  - joblib==1.0.1
- numpy=1.19.5
- pandas=1.2.4
- scikit-learn=0.24.2
channels:
- anaconda
- conda-forge



In [ ]:
# Configuración del servicio web con más recursos
aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1,     
    memory_gb=2,      
    description="API para predecir bancarrota empresarial",
    auth_enabled=True  
)

print("Desplegando modelo como servicio web...")
print("Este proceso puede tardar varios minutos...")

# Desplegar el modelo
try:
    service = Model.deploy(
        workspace=ws,
        name='bankruptcy-prediction-diego', 
        models=[registered_model],
        inference_config=inference_config,
        deployment_config=aci_config,
        overwrite=True,
    )
    
    # Esperar a que el servicio se despliegue
    print("Esperando a que el servicio se despliegue...")
    service.wait_for_deployment(show_output=True)
    
    # Obtener la URI del servicio
    scoring_uri = service.scoring_uri
    print(f"¡Servicio desplegado exitosamente!")
    print(f"URI del servicio: {scoring_uri}")
    
except Exception as e:
    print(f"Error al desplegar el servicio: {e}")
    print("Revisa los registros de Azure para más detalles.")
    exit(1)

Desplegando modelo como servicio web...
Este proceso puede tardar varios minutos...


C:\Users\A\AppData\Local\Temp\ipykernel_15264\2411632133.py:14: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


Esperando a que el servicio se despliegue...
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-04-03 19:31:09-06:00 Creating Container Registry if not exists..
2025-04-03 19:41:09-06:00 Registering the environment..
2025-04-03 19:41:11-06:00 Building image..
2025-04-03 19:53:10-06:00 Generating deployment configuration.
2025-04-03 19:53:12-06:00 Submitting deployment to compute..
2025-04-03 19:53:22-06:00 Checking the status of deployment bankruptcy-prediction-diego..
2025-04-03 19:54:41-06:00 Checking the status of inference endpoint bankruptcy-prediction-diego.
Succeeded
ACI service creation operation finished, operation "Succeeded"
¡Servicio desplegado exitosamente!
URI del servicio: http://63282996-37d4-43dc-bf14-fce6f11b4078.centralindia.azurecontainer.io/score


In [17]:
try:
    # Obtener y mostrar los logs para diagnóstico
    print("\nLogs del servicio desplegado:")
    logs = service.get_logs()
    print(logs)
    
    # Guardar la URI en un archivo JSON
    scoreuri = json.dumps({"URI": [scoring_uri]})
    with open("uri.json", "w") as file:
        file.write(scoreuri)
    
    # Guardar las credenciales de autenticación
    if service.auth_enabled:
        print("El servicio requiere autenticación.")
        print(f"Clave principal: {service.get_keys()[0]}")
        # Guardar la clave para uso futuro
        with open("api_key.json", "w") as key_file:
            json.dump({"key": service.get_keys()[0]}, key_file)
            print("Clave de API guardada en 'api_key.json'")
    
    print("""
    Para usar la API:
    1. Ejecuta el script 'api.py' para realizar predicciones
    2. O utiliza la URI con tu propia aplicación
    """)
except Exception as e:
    print(f"Error al obtener información del servicio: {e}")


Logs del servicio desplegado:
/bin/bash: /azureml-envs/azureml_605d10292a84721aabef4fa7b6aba91d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_605d10292a84721aabef4fa7b6aba91d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_605d10292a84721aabef4fa7b6aba91d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_605d10292a84721aabef4fa7b6aba91d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2025-04-04T01:54:34,955059865+00:00 - iot-server/run 
2025-04-04T01:54:34,956937858+00:00 - rsyslog/run 
bash: /azureml-envs/azureml_605d10292a84721aabef4fa7b6aba91d/lib/libtinfo.so.6: no version information available (required by bash)
2025-04-04T01:54:34,975556277+00:00 - nginx/run 
2025-04-04T01:54:34,986656161+00:00 - gunicorn/run 
2025-04-04T01:54:34,990140961+00:00 | gunicorn/run | 
2025-04-0